In [42]:
import csv
import json
import requests
import urllib.parse

In [19]:
api_url = "https://geocoder.api.gov.bc.ca/"

In [37]:
api = "addresses.json?addressString=%s"

In [38]:
query = "525 Superior street, Victoria BC"

In [39]:
quoted_query = urllib.parse.quote_plus(query)
quoted_query

'525+Superior+street%2C+Victoria+BC'

In [40]:
resp = requests.get(api_url + api % quoted_query)

In [61]:
parsed = json.loads(resp.text)
print(type(parsed))
#print(json.dumps(parsed, indent=4, sort_keys=True))
print(parsed['features'][0]['properties']['score'])
print(parsed['features'][0]['properties']['matchPrecision'])
print(parsed['features'][0]['properties']['fullAddress'])
print(parsed['features'][0]['properties']['faults'])


<class 'dict'>
100
CIVIC_NUMBER
525 Superior St, Victoria, BC
[]


In [27]:
api_url + api % "525 supriour street"

'https://geocoder.api.gov.bc.ca/address.json?addressString=525 supriour street'

In [ ]:
'https://geocoder.api.gov.bc.ca/addresses.json?addressString=25%20Superior%20Street%2C%20Victoria%2C%20BC&locationDescriptor=any&maxResults=1&interpolation=adaptive&echo=true&brief=false&autoComplete=false&setBack=0&outputSRS=4326&minScore=1&provinceCode=BC&apikey=6097f62f6a8144edae53c59fc7c12351'

In [114]:
with open('atp_addresses.csv', encoding = "ISO-8859-1") as atp_file:
    atp_tests = csv.reader(atp_file, delimiter=',', quotechar='"') 
    headder_row = next(atp_tests)
    
    count = int(2) # starting at 2 as the CSV has a header and the ID for that row is 1 not 0
    faults = int()
    
    NAME = 0
    QUERY = 1
    EXPECTED_MATCH_PRECISION = 2
    EXPECTTED_FULL_ADDRESS = 3
    EXPECTED_FAULTS = 4
    
    for row in atp_tests:

        quoted_query = urllib.parse.quote_plus(row[QUERY])

        resp = requests.get(api_url + api % quoted_query)
        parsed = json.loads(resp.text)
                
        if parsed['features'][0]['properties']['matchPrecision'] != row[EXPECTED_MATCH_PRECISION]:
            faults+=1
            print("TEST CASE: %d, ID: %s, QUERY: \"%s\"" % (count, row[name], row[query]))
            print("MATCH PRECISION : EXPECTED: %s, GOT: %s" % (row[EXPECTED_MATCH_PRECISION], parsed['features'][0]['properties']['matchPrecision']))

        if parsed['features'][0]['properties']['fullAddress'] != row[EXPECTTED_FULL_ADDRESS]:
            faults+=1
            print("TEST CASE: %d, ID: %s, QUERY: \"%s\"" % (count, row[name], row[query]))
            print("FULL ADDRESS EXPECTED : %s, GOT: %s" % (row[EXPECTTED_FULL_ADDRESS], parsed['features'][0]['properties']['fullAddress']))

            
        if str(parsed['features'][0]['properties']['faults']) != row[EXPECTED_FAULTS]:  
            faults+=1
            print("TEST CASE: %d, ID: %s, QUERY: \"%s\"" % (count, row[name], row[query]))
            print("FAULTS : EXPECTED: %s, GOT: %s" % (row[EXPECTED_FAULTS], str(parsed['features'][0]['properties']['faults'])))

        count+=1
    print ("Total test cases: %d" % count)
    print ("Faults: %d" % faults)
    print ("count - faults: %d" % (count - faults))


TEST CASE: 10, ID: CivicAddressWithAbbreviatedStreetName, QUERY: "3928 Cedar Hill X Rd	 Saanich	 BC"
MATCH PRECISION : EXPECTED: STREET, GOT: CIVIC_NUMBER
TEST CASE: 11, ID: CivicAddressWithAbbreviatedLocalityName, QUERY: "7495 Columbia Ave	 Radium Hot Sprgs	 BC"
MATCH PRECISION : EXPECTED: STREET, GOT: CIVIC_NUMBER
TEST CASE: 12, ID: civicAddressWithoutProvinceCode, QUERY: "2050 SW Marine Dr	 Vancouver"
FAULTS : EXPECTED: [PROVINCE.missing:1], GOT: [{'value': '', 'element': 'PROVINCE', 'fault': 'missing', 'penalty': 1}]
TEST CASE: 14, ID: civicAddressWithoutUnit, QUERY: "10372 99 St	 Taylor	BC"
FULL ADDRESS EXPECTED : 10372 99th St, Taylor, BC, GOT: 10372 99 St, Taylor, BC
TEST CASE: 19, ID: civicAddressWithUnitandUnknownSiteName, QUERY: "Douglas College	 Unit 170 -- 22470 Dewdney Trunk Rd	 Maple Ridge	 BC"
FAULTS : EXPECTED: [SITE_NAME.notMatched], GOT: [{'value': 'DOUGLAS COLLEGE', 'element': 'SITE_NAME', 'fault': 'notMatched', 'penalty': 10}, {'value': '', 'element': 'MAX_RESULTS',

TEST CASE: 46, ID: CivicAddressWrongCharacterInStreetName, QUERY: "30 Fifth Cres	 Mackenzie	 BC"
FAULTS : EXPECTED: [STREET_NAME.spelledWrong:2], GOT: [{'value': 'FIFTH', 'element': 'STREET_NAME', 'fault': 'spelledWrong', 'penalty': 2}]
TEST CASE: 47, ID: CivicAddressWrongCharacterInLocality, QUERY: "30 Firth Cres	 Mackensie	 BC"
FAULTS : EXPECTED: [LOCALITY.spelledWrong:2], GOT: [{'value': 'MACKENSIE', 'element': 'LOCALITY', 'fault': 'spelledWrong', 'penalty': 2}]
TEST CASE: 48, ID: CivicAddressMissingWordInSiteName, QUERY: "Anahim Airport -- Anahim Lake	 BC"
FAULTS : EXPECTED: [SITE_NAME.partialMatch:2], GOT: [{'value': 'ANAHIM AIRP', 'element': 'SITE_NAME', 'fault': 'partialMatch', 'penalty': 2}]
TEST CASE: 49, ID: CivicAddressWrongWordInStreetName, QUERY: "6006 Happy Hilltop RD	 Summerland	BC"
FAULTS : EXPECTED: [UNRECOGNIZED.notAllowed:33	 STREET_NAME.partialMatch:1], GOT: [{'value': 'HILLTOP RD', 'element': 'LOCALITY_GARBAGE', 'fault': 'notAllowed', 'penalty': 3}, {'value': '', '

TEST CASE: 72, ID: CivicAddressUnknownStreetType, QUERY: "561 60 Roseway SE	 Salmon Arm	 BC "
FAULTS : EXPECTED: [UNRECOGNIZED.notAllowed:33	 STREET_TYPE.missing:6], GOT: [{'value': 'ROSEWAY SE', 'element': 'LOCALITY_GARBAGE', 'fault': 'notAllowed', 'penalty': 3}, {'value': '', 'element': 'STREET_DIRECTION', 'fault': 'missing', 'penalty': 2}, {'value': '', 'element': 'STREET_TYPE', 'fault': 'missing', 'penalty': 6}]
TEST CASE: 73, ID: CivicAddressUnknownStreetName, QUERY: "561 Gruffle St SE	 Salmon Arm	 BC"
MATCH PRECISION : EXPECTED: BLOCK, GOT: LOCALITY
TEST CASE: 73, ID: CivicAddressUnknownStreetName, QUERY: "561 Gruffle St SE	 Salmon Arm	 BC"
FULL ADDRESS EXPECTED : 561 Salmon Rd, Central Saanich, BC, GOT: Salmon Arm, BC
TEST CASE: 73, ID: CivicAddressUnknownStreetName, QUERY: "561 Gruffle St SE	 Salmon Arm	 BC"
FAULTS : EXPECTED: [UNRECOGNIZED.notAllowed:33	 LOCALITY.missing:10	 STREET_DIRECTION.notMatched:2	 STREET_TYPE.notMatched:3	 STREET_QUALIFIER.notMatched:1	 MAX_RESULTS.too

TEST CASE: 101, ID: CivicAddressWithUnabbreviatedFloorDesignator, QUERY: "Floor 2	 1175 Douglas St	 Victoria	 BC"
MATCH PRECISION : EXPECTED: CIVIC_NUMBER, GOT: UNIT
TEST CASE: 101, ID: CivicAddressWithUnabbreviatedFloorDesignator, QUERY: "Floor 2	 1175 Douglas St	 Victoria	 BC"
FULL ADDRESS EXPECTED : FLR 2 -- 1175 Douglas St, Victoria, BC, GOT: UNIT 2 -- 1175 Douglas St, Victoria, BC
TEST CASE: 101, ID: CivicAddressWithUnabbreviatedFloorDesignator, QUERY: "Floor 2	 1175 Douglas St	 Victoria	 BC"
FAULTS : EXPECTED: [UNIT_DESIGNATOR.notMatched:1	 UNIT_NUMBER.notMatched:1], GOT: [{'value': 'FLR', 'element': 'UNIT_DESIGNATOR', 'fault': 'isAlias', 'penalty': 0}]
TEST CASE: 102, ID: CivicAddressWithReverseUnabbreviatedFloorDesignator, QUERY: "2nd Floor	 1175 Douglas St	 Victoria	 BC"
MATCH PRECISION : EXPECTED: CIVIC_NUMBER, GOT: UNIT
TEST CASE: 102, ID: CivicAddressWithReverseUnabbreviatedFloorDesignator, QUERY: "2nd Floor	 1175 Douglas St	 Victoria	 BC"
FULL ADDRESS EXPECTED : FLR 2 -- 1

TEST CASE: 135, ID: CivicAddressFailedInPast, QUERY: "gorge and gorge"
FAULTS : EXPECTED: [STREET_QUALIFIER.missing:1	 LOCALITY.missing:10	 PROVINCE.missing:1], GOT: [{'value': '', 'element': 'STREET_QUALIFIER', 'fault': 'missing', 'penalty': 1}, {'value': '', 'element': 'LOCALITY', 'fault': 'missing', 'penalty': 10}, {'value': '', 'element': 'PROVINCE', 'fault': 'missing', 'penalty': 1}]
TEST CASE: 137, ID: CivicAddressNotInAnyAddressRange, QUERY: "1 Commercial Dr	 Dease Lake	 BC"
MATCH PRECISION : EXPECTED: BLOCK, GOT: STREET
TEST CASE: 137, ID: CivicAddressNotInAnyAddressRange, QUERY: "1 Commercial Dr	 Dease Lake	 BC"
FULL ADDRESS EXPECTED : 1 Commercial Dr, Dease Lake, BC, GOT: Commercial Dr, Dease Lake, BC
TEST CASE: 137, ID: CivicAddressNotInAnyAddressRange, QUERY: "1 Commercial Dr	 Dease Lake	 BC"
FAULTS : EXPECTED: [], GOT: [{'value': 1, 'element': 'CIVIC_NUMBER', 'fault': 'notInAnyBlock', 'penalty': 1}]
TEST CASE: 138, ID: CivicAddressStreetNameNotPrimary, QUERY: "1271 Winches